# 训练领域模型--医疗模型

分三阶段：

- 第一阶段：PT(continue PreTraining)增量预训练，在海量领域文档数据上二次预训练LLaMA模型，如有需要可以扩充领域词表，比如医疗领域词表；
- 第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在第一阶段的预训练模型基础上做指令精调；
- 第三阶段：RM(Reward Model)奖励模型，和基于人类反馈的强化学习(RLHF), 此本部分见[shibing624/MedicalGPT](https://github.com/shibing624/MedicalGPT)。


## PT(continue PreTraining)增量预训练

使用百科类文档类数据集，用来在领域数据集上增量预训练或二次预训练，期望能把领域知识注入给模型，以医疗领域为例，希望增量预训练，能让模型理解感冒的症状、病因、治疗药品、治疗方法、药品疗效等知识，便于后续的SFT监督微调能激活这些内在知识。

这里说明一点，像GPT3、LLaMA这样的大模型理论上是可以从增量预训练中获益，但增量预训练需要满足两个要求：1）高质量的预训练样本；2）较大的计算资源，显存要求高，即使是用LoRA技术，也要满足block_size=1024或2048长度的文本加载到显存中。

其次，如果你的项目用到的数据是模型预训练中已经使用了的，如维基百科、ArXiv等LLaMA模型预训练用了的，则这些数据是没有必要再喂给LLaMA增量预训练，而且预训练样本的质量如果不够高，也可能会损害原模型的生成能力。

tips：PT阶段是可选项，请慎重处理。

In [ ]:
!git clone --depth 1 https://github.com/shibing624/textgen.git
%cd textgen
%ls

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
%cd examples/llama

In [ ]:
%pwd

In [ ]:
import os
import sys

import pandas as pd
from datasets import load_dataset, load_from_disk
from loguru import logger

sys.path.append('../..')
from textgen import LlamaModel


In [ ]:
logger.info('Loading data...')
model_args = {
    "is_pretraining": True,
    "block_size": 1024,
    "use_peft": True,
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "per_device_train_batch_size": 2,
    "eval_batch_size": 2,
    "num_train_epochs": 1,
    "output_dir": './outputs-pretraining-medical/',
    "resume_from_checkpoint": './outputs-pretraining-medical/',
    "eval_steps": 50,
    "save_steps": 100,
}
# 此处以llama-plus为例，效果较好的中文模型如下：
# 7b: minlik/chinese-llama-plus-7b-merged
# 13b: shibing624/chinese-llama-plus-13b-hf
model = LlamaModel('llama', 'shibing624/chinese-llama-plus-13b-hf', args=model_args)
model

In [ ]:
print(model.predict(["给出三个保持健康的秘诀。"]))
print(model.predict(["乙肝和丙肝的区别"]))
print(model.predict(["什么补肾最好"]))

In [ ]:
def load_pt_data(data, train_limit_size=500):
    if data.endswith('.json') or data.endswith('.jsonl'):
        dataset = load_dataset("json", data_files=data)
    elif os.path.isdir(data):
        dataset = load_from_disk(data)
    else:
        dataset = load_dataset(data, 'pretrain')
    logger.debug(dataset)

    if train_limit_size > 0:
        dataset_sample = dataset["train"].shuffle(seed=42).select(range(train_limit_size))
    else:
        dataset_sample = dataset["train"].shuffle(seed=42)
    all_datasets = dataset_sample
    logger.debug(f"all_datasets size:{len(all_datasets)}, first line: {next(iter(all_datasets))}")
    data = []
    for example in all_datasets:
        data.append(example['text'])
    logger.info(f"total data size:{len(data)}, head: {data[:3]}")
    return data

In [ ]:
train_data = load_pt_data('shibing624/medical', 500)
logger.debug(f'train_data, size: {len(train_data)}, head top3: {train_data[:3]}')
train_df = pd.DataFrame(train_data, columns=["text"])
train_df

In [ ]:
eval_df = train_df[:10]
eval_df

In [ ]:
train_df = train_df[10:]
train_df

开始训练啦，我本地测试了下，block_size=1024, batch_size=2, peft_type=lora，model=llama-13b, 开fp16, 显存占用30G。

- 如果你的显存不足，可以改小batch_size=1, block_size=512（影响训练的上下文最大长度）,peft_type=adalora;
- 如果你的显存更大，可以改大block_size=2048, 此为llama原始预训练长度，不能更大啦；调大batch_size。

In [ ]:
model.train_model(train_df, eval_data=eval_df)

In [ ]:
print(model.predict(["给出三个保持健康的秘诀。"]))
print(model.predict(["乙肝和丙肝的区别"]))
print(model.predict(["什么补肾最好"]))

以下脚本是用lora训练时需要执行的，用来把lora权重merge到base model中，每一阶段都需要执行此merge操作。

合并lora模型的权重参数到base model，

细节可以参考合并脚本：textgen/llama/merge_peft_adapter.py

In [ ]:
from transformers import LlamaForCausalLM
output_dir = "merged_pt"
base_model = model.model.merge_and_unload()
base_model

In [ ]:
model.tokenizer.save_pretrained(output_dir)
LlamaForCausalLM.save_pretrained(base_model, output_dir)

In [ ]:
%ls merged_pt

## SFT(Supervised Fine-tuning)有监督微调

构造指令数据集，在第一阶段的预训练模型基础上做指令精调。

如果是LoRA的增量预训练，需要先把LoRA权重merge到base model, 使用merge脚本完成合并操作，再SFT；
如果没有做增量预训练，则直接在底座模型上SFT就可以。


<img src=../../docs/rlhf.png height=500 />


基于llama-13b-plus底座模型或者merge_pt的底座模型，SFT监督训练生成llama-13b-sft:

In [ ]:
from datasets import load_dataset, concatenate_datasets

def load_sft_data(debug=True):
    raw_dataset = load_dataset('shibing624/medical', 'finetune')
    logger.debug(raw_dataset)

    all_datasets = []
    if debug:
        all_datasets.append(raw_dataset["train"].shuffle().select(range(1000)))
    else:
        all_datasets.append(raw_dataset["train"])
    # sft 阶段，除了加载医疗训练集外，为了避免模型灾难遗忘，需要强化其正常对话能力，此处加载中英文alpaca、中文belle数据集
    dataset_names = ["shibing624/alpaca-zh", "yahma/alpaca-cleaned", "BelleGroup/train_0.5M_CN", "BelleGroup/generated_chat_0.4M"]
    for name in dataset_names:
        raw_dataset = load_dataset(name)
        ds = raw_dataset["train"]
        if debug:
            ds = ds.shuffle().select(range(1000))
        all_datasets.append(ds)
    all_datasets = concatenate_datasets(all_datasets)
    data = []
    logger.debug(f"all_datasets size:{len(all_datasets)}, first line: {next(iter(all_datasets))}")
    for example in all_datasets:
        instruction, intput_str, output_str = example["instruction"], example["input"], example["output"]
        data.append([instruction, intput_str, output_str])
    logger.info(f"total data size:{len(data)}, head: {data[:3]}")
    return data



In [ ]:
model_name = 'merged_pt' if os.path.exists('merged_pt') else 'shibing624/chinese-llama-plus-13b-hf'

In [ ]:
# fine-tune Llama model
model_args = {
    "use_peft": True,
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "max_length": 256,
    "per_device_train_batch_size": 8,
    "eval_batch_size": 8,
    "num_train_epochs": 0.5,
    "is_train_on_prompt": False,
    "output_dir": './outputs-sft/',
    "resume_from_checkpoint": './outputs-llama-13b-sft/',
    "eval_steps": 100,
    "save_steps": 500,
}
model = LlamaModel('llama', model_name, args=model_args)
model

In [ ]:
print(model.predict(["给出三个保持健康的秘诀。"]))
print(model.predict(["乙肝和丙肝的区别"]))
print(model.predict(["什么补肾最好"]))

In [ ]:
train_data = load_sft_data(debug=True)
logger.debug('train_data: {}'.format(train_data[:10]))
train_df = pd.DataFrame(train_data, columns=["instruction", "input", "output"])
train_df

In [ ]:
eval_df = train_df[:10]
train_df = train_df[10:]
eval_df

In [ ]:
model.train_model(train_df, eval_data=eval_df)

In [ ]:
def generate_prompt(instruction):
    return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n\n"""

print(model.predict([generate_prompt("给出三个保持健康的秘诀。")]))
print(model.predict([generate_prompt("乙肝和丙肝的区别")]))
print(model.predict(["什么补肾最好"]))
print(model.predict([generate_prompt("防中风还可吃什么药？")]))

In [ ]:
%ls outputs-llama-13b-sft

把lora权重merge到base model中:

In [ ]:
output_dir = "merged_sft"
base_model = model.model.merge_and_unload()
model.tokenizer.save_pretrained(output_dir)
LlamaForCausalLM.save_pretrained(base_model, output_dir)

In [ ]:
%ls merged_sft

注意：使用lora训练，每一步的lora权重都要合并到base模型中，再进行下一步。